# Imports

In [1]:
from PAL import *
pd.set_option('display.max_columns', None)

engine = Redshift_Connection('ssopariwalla')
schema = 'halo_connect_customer_data'
CIP_schema = CIP_account_schema_mapping['AMZ']
gemini_schema = "prod-gem-plat-event-database"

In [2]:
FACs_to_exclude = ('Y87BQV9', #AperiaTech
                   'X77OKO7', # Amazon Last Mile
                   '1L10O39N', # Gemini DVT Deployments
                   '1GOPAPMF', # Not Configured
                   '3FF99DS',  # UNKNOWN_FLEET
                   '1FBR7DGI', # TTN
                   '1M5190AC',  # YTX
                   '19RVO6ZP' #Amazon EOL Vehicles	
                   )

# Issue Escalation

In [3]:
query = f"""
WITH indexed_events AS (
    SELECT
        event_table.event_id,
        event_table.unique_id,
        event_table.event_type,
        event_table.pressure_date AS event_table__pressure_date,
        event_table.ts_created AS event_table__ts_created,
        RANK() OVER (PARTITION BY event_table.event_id ORDER BY event_status_id ASC) as event_update_rank_ASC,
        RANK() OVER (PARTITION BY event_table.event_id ORDER BY event_status_id DESC) as event_update_rank_DESC, 
        event_status.event_status_id,
        event_status.ts_created AS event_status__ts_created,
        event_status.severity,
        event_status.status,
        event_status.pressure_date AS event_status__pressure_date,
        RANK() OVER (PARTITION BY event_table.event_id, meta_data.unique_id ORDER BY meta_data.id DESC) as meta_data_id_rank_desc,
        meta_data.side,
        meta_data.axle,
        meta_data.position,
        meta_data.sensor_number,
        fleet_meta_data.fleet_name,
        vehicle_meta_data.fleet_vehicle_id,
        vehicle_groups.name as vehicle_group
    FROM
        {schema}.event_table INNER JOIN {schema}.event_status
            ON event_table.event_id = event_status.event_id
        INNER JOIN {schema}.meta_data
            ON (meta_data.unique_id = event_table.unique_id
                AND meta_data.created_at < event_table.pressure_date
                AND (meta_data.deactivated_at > event_table.pressure_date OR meta_data.deactivated_at IS NULL))
        INNER JOIN {schema}.vehicle_meta_data
            ON (vehicle_meta_data.vehicle_id = meta_data.vehicle_id
                AND vehicle_meta_data.created_at < event_table.pressure_date
                AND (vehicle_meta_data.retired_at > event_table.pressure_date OR vehicle_meta_data.retired_at IS NULL))
        INNER JOIN {schema}.fleet_meta_data
            ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
        LEFT JOIN {schema}.vehicle_groups_vehicles
            ON vehicle_meta_data.vehicle_id = vehicle_groups_vehicles.vehicle_id
        LEFT JOIN {schema}.vehicle_groups ON vehicle_groups_vehicles.vehicle_group_id = vehicle_groups.id 
    WHERE
        event_table.event_type != 'MAINT'
        AND event_table.ts_created >= date_trunc('year', CURRENT_DATE)::date
        AND fleet_meta_data.code NOT IN {FACs_to_exclude}
        AND fleet_meta_data.status = 'ACTIVE'
),
indexed_sensor_data AS (
    SELECT
        unique_id,
        pressure_date,
        pressure,
        speed,
        latitude,
        longitude,
        ROW_NUMBER() OVER(PARTITION BY unique_id, pressure_date ORDER BY speed ASC, pressure DESC) as row_number
    FROM
        {schema}.sensor_data
    WHERE
        pressure_date >= date_trunc('year', CURRENT_DATE)::date
)
SELECT DISTINCT
    event_id,
    event_table__ts_created as ts_created,
    MIN(CASE WHEN event_update_rank_DESC = 1 THEN severity END) AS current_severity,
    MIN(CASE WHEN event_update_rank_DESC = 1 THEN indexed_events.status END) AS current_status,
    event_type,
    event_table__pressure_date AS pressure_date,
    side,
    axle,
    position,
    sensor_number,
    fleet_name,
    CASE
        WHEN fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END as maint_provider,
    vehicle_group,
    fleet_vehicle_id,
    CASE
        WHEN COUNT(DISTINCT severity) > 1 THEN 'Y'
        ELSE 'N'
    END AS escalated,
    MIN(CASE WHEN severity = 'MINOR' THEN event_status__ts_created END) as minor_ts,
    COALESCE(
        EXTRACT(EPOCH FROM MIN(CASE WHEN severity = 'MAJOR' THEN event_status__ts_created END) - MIN(CASE WHEN severity = 'MINOR' THEN event_status__ts_created END)) / 86400.0,
        EXTRACT(EPOCH FROM MIN(CASE WHEN severity = 'CRITICAL' THEN event_status__ts_created END) - MIN(CASE WHEN severity = 'MINOR' THEN event_status__ts_created END)) / 86400.0,
        EXTRACT(EPOCH FROM MIN(CASE WHEN indexed_events.status = 'CLOSED' THEN event_status__ts_created END) - MIN(CASE WHEN severity = 'MINOR' THEN event_status__ts_created END)) / 86400.0
    ) as minor_duration,
    MIN(CASE WHEN severity = 'MAJOR' THEN event_status__ts_created END) as major_ts,
    COALESCE(
        EXTRACT(EPOCH FROM MIN(CASE WHEN severity = 'CRITICAL' THEN event_status__ts_created END) - MIN(CASE WHEN severity = 'MAJOR' THEN event_status__ts_created END)) / 86400.0,
        EXTRACT(EPOCH FROM MIN(CASE WHEN indexed_events.status = 'CLOSED' THEN event_status__ts_created END) - MIN(CASE WHEN severity = 'MAJOR' THEN event_status__ts_created END)) / 86400.0
    ) as major_duration,
    MIN(CASE WHEN severity = 'CRITICAL' THEN event_status__ts_created END) as critical_ts,
    EXTRACT(EPOCH FROM MIN(CASE WHEN indexed_events.status = 'CLOSED' THEN event_status__ts_created END) - MIN(CASE WHEN severity = 'CRITICAL' THEN event_status__ts_created END)) / 86400.0 as critical_duration,
    MIN(CASE WHEN event_update_rank_ASC = 1 THEN severity END) AS first_severity,
    MIN(CASE WHEN event_update_rank_DESC = 1 THEN severity END) AS last_severity,
    MAX(event_status__ts_created) AS last_change_ts,
    EXTRACT(EPOCH FROM MAX(CASE WHEN indexed_events.status = 'CLOSED' THEN event_status__ts_created END) - MIN(CASE WHEN indexed_events.status = 'CLOSED' THEN event_table__ts_created END)) / 86400.0 AS time_to_resolution,
    CASE
        WHEN MAX(CASE WHEN event_update_rank_DESC = 1 THEN indexed_events.status END) = 'OPEN' THEN
            EXTRACT(EPOCH FROM CURRENT_TIMESTAMP - MIN(event_status__ts_created)) / 86400.0
    END AS open_age,
    isd.pressure AS tire_pressure_at_detection,
    CASE 
      WHEN isd.speed IS NOT NULL THEN isd.latitude || ', ' || isd.longitude 
      ELSE NULL 
    END AS location_at_detection
FROM 
    indexed_events LEFT JOIN (SELECT * FROM indexed_sensor_data WHERE row_number = 1) isd
        ON (indexed_events.event_table__pressure_date = isd.pressure_date
            AND indexed_events.unique_id = isd.unique_id)
WHERE
    meta_data_id_rank_desc = 1
GROUP BY
    event_id,
    isd.pressure,
    isd.speed, isd.latitude, isd.longitude,
    event_table__ts_created,
    event_type,
    event_table__pressure_date,
    side,
    axle,
    position,
    sensor_number,
    fleet_name,
    CASE
        WHEN fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END,
    vehicle_group,
    fleet_vehicle_id
"""

with open("amz_dashboard_alerts_data_query.txt", "w") as file:
    file.write(query)


In [4]:
# issue_escalation = pd.read_sql(query,engine)

# Hardware Report

In [5]:
query = f"""
WITH vehicle_list AS (
    SELECT
        fleet_meta_data.fleet_name,
        vehicle_groups.name as vehicle_group,
        vehicle_meta_data.vehicle_id,
        vehicle_meta_data.fleet_vehicle_id,
        vehicle_meta_data.vehicle_type,
        vehicle_meta_data.vin_number
    FROM
        {schema}.fleet_meta_data INNER JOIN {schema}.vehicle_meta_data
            ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
        LEFT JOIN {schema}.vehicle_groups_vehicles
            ON vehicle_meta_data.vehicle_id = vehicle_groups_vehicles.vehicle_id
        LEFT JOIN {schema}.vehicle_groups 
            ON vehicle_groups_vehicles.vehicle_group_id = vehicle_groups.id 
    WHERE
        fleet_meta_data.status = 'ACTIVE'
        AND vehicle_meta_data.archived = 0
        AND fleet_meta_data.code NOT IN {FACs_to_exclude}
        AND fleet_meta_data.status = 'ACTIVE'
),
sensor_analytics AS (
    SELECT
        vehicle_meta_data.vehicle_id,
        meta_data.side || meta_data.axle || meta_data.position as tire,
        MAX(pressure_date) as last_sensor_transmission,
        CURRENT_TIMESTAMP - MAX(pressure_date) as time_since_last_sensor_transmission
    FROM
        {schema}.vehicle_meta_data INNER JOIN {schema}.meta_data
            ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
        INNER JOIN {schema}.sensor_data
            ON meta_data.id = sensor_data.meta_data_id
    WHERE
        (CASE WHEN vehicle_meta_data.vehicle_type = 'Tractor Tandem-Dual' THEN meta_data.axle <= 3 ELSE 1=1 END)
        AND vehicle_meta_data.archived = 0
    GROUP BY
        vehicle_meta_data.vehicle_id,
        meta_data.side || meta_data.axle || meta_data.position
),
gw_analytics AS (
    SELECT
        vehicle_meta_data.vehicle_id,
        gateway_meta_data.gateway_id,
        gateway_meta_data.type as gateway_type,
        CASE
            WHEN gateway_meta_data.type = 'APOLLO' THEN MAX(file_meta_data.file_upload_time)
            WHEN gateway_meta_data.type = 'GEMINI' THEN TIMESTAMP 'epoch' + (MAX(heartbeat.event_timestamp) / 1000) * INTERVAL '1 second'
        END as last_gw_transmission
    FROM
        {schema}.vehicle_meta_data INNER JOIN {schema}.gateway_meta_data
            ON vehicle_meta_data.cycle_number = gateway_meta_data.cycle_number
        LEFT JOIN {schema}.file_meta_data
            ON file_meta_data.cycle_number = gateway_meta_data.cycle_number
        LEFT JOIN (SELECT * FROM "{gemini_schema}"."prod-gem-plat-event-table" WHERE measure_name = 'heartbeat') heartbeat
            ON heartbeat.device_id= gateway_meta_data.gateway_id
    WHERE
        file_meta_data.id IN (SELECT MAX(id) FROM {schema}.file_meta_data GROUP BY cycle_number)
    GROUP BY
        vehicle_meta_data.vehicle_id,
        gateway_meta_data.gateway_id,
        gateway_meta_data.type
)
SELECT
    vehicle_list.fleet_name,
    CASE
        WHEN fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END AS maint_provider,
    vehicle_list.vehicle_group,
    vehicle_list.vehicle_id,
    vehicle_list.fleet_vehicle_id,
    vehicle_list.vehicle_type,
    vehicle_list.vin_number,
    MAX(gw_analytics.last_gw_transmission) last_gw_transmission,
    CASE
        WHEN (CURRENT_TIMESTAMP - MAX(gw_analytics.last_gw_transmission) >= INTERVAL '2 DAY') OR (MAX(gw_analytics.last_gw_transmission) IS NULL) THEN 'OFFLINE'
        ELSE 'ONLINE'
    END AS gateway_status,
    SUM(
        CASE
            WHEN sensor_analytics.time_since_last_sensor_transmission >= INTERVAL '2 DAY' THEN 1
            ELSE 0
        END
    ) AS number_offline_sensors
FROM
    vehicle_list LEFT JOIN sensor_analytics
        ON vehicle_list.vehicle_id = sensor_analytics.vehicle_id
    LEFT JOIN gw_analytics
        ON gw_analytics.vehicle_id = vehicle_list.vehicle_id
GROUP BY
    vehicle_list.fleet_name,
    CASE
        WHEN fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END,
    vehicle_list.vehicle_group,
    vehicle_list.vehicle_id,
    vehicle_list.fleet_vehicle_id,
    vehicle_list.vehicle_type,
    vehicle_list.vin_number
"""
with open("amz_dashboard_hardware_status_query.txt", "w") as file:
    file.write(query)

In [6]:
# hardware_status = pd.read_sql(query,engine)

# Get Number of Closed Alerts per Day YTD

In [7]:
query = f"""
WITH closed_alerts AS (
    SELECT
        event_status.event_id,
        event_status.ts_created::date AS closed_date,
        event_status.severity,
        fleet_meta_data.fleet_name,
        vehicle_groups.name AS vehicle_group,
        CASE 
            WHEN fleet_meta_data.fleet_name = 'TOMY' THEN 'TOMY'
            ELSE 'AFP'
        END AS maint_provider
    FROM
        {schema}.event_table
    INNER JOIN {schema}.event_status
        ON event_table.event_id = event_status.event_id
    INNER JOIN {schema}.meta_data
        ON meta_data.unique_id = event_table.unique_id
    INNER JOIN {schema}.vehicle_meta_data
        ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
    INNER JOIN {schema}.fleet_meta_data
        ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
    LEFT JOIN {schema}.vehicle_groups_vehicles
        ON vehicle_meta_data.vehicle_id = vehicle_groups_vehicles.vehicle_id
    LEFT JOIN {schema}.vehicle_groups
        ON vehicle_groups_vehicles.vehicle_group_id = vehicle_groups.id
    WHERE
        event_table.event_type != 'MAINT'
        AND meta_data.id IN (
            SELECT MAX(id)
            FROM {schema}.meta_data
            GROUP BY unique_id
        )
        AND event_table.ts_created >= date_trunc('year', CURRENT_DATE)::date
        AND fleet_meta_data.code NOT IN {FACs_to_exclude}
        AND fleet_meta_data.status = 'ACTIVE'
        AND event_status.event_status_id IN (
            SELECT MAX(event_status_id)
            FROM {schema}.event_status
            WHERE status = 'CLOSED'
            GROUP BY event_id
        )


)
SELECT
    closed_date,
    severity,
    fleet_name,
    vehicle_group,
    maint_provider,
    COUNT(DISTINCT event_id) AS num_alerts_closed
FROM
    closed_alerts
GROUP BY
    closed_date,
    severity,
    fleet_name,
    vehicle_group,
    maint_provider
ORDER BY
    closed_date
"""

with open("amz_dashboard_alert_closure_counts.txt", "w") as file:
    file.write(query)


In [ ]:
# closed_alerts_info = pd.read_sql(query,engine)


# EOL Manifest

In [8]:
query = f"""
SELECT
    code,
    fleet_name,
    vehicle_id,
    fleet_vehicle_id,
    archived
FROM
    {schema}.fleet_meta_data INNER JOIN {schema}.vehicle_meta_data
        ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
WHERE
    fleet_meta_data.code = '19RVO6ZP' -- EOL
"""
with open("amz_dashboard_EOL_manifest.txt", "w") as file:
    file.write(query)

In [9]:
# EOL_manifest = pd.read_sql(query,engine)

# CIP Data

In [11]:
query = f"""    
SELECT
    fleet_meta_data.fleet_name,
    vehicle_groups.name AS vehicle_group,
    CASE 
        WHEN fleet_meta_data.fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END AS maint_provider,
    cold_inflation_pressure.time as date,
    CASE
      WHEN sensor_id ~ '_2$' OR sensor_id ~ '_3$' THEN 'Gen 2 Inflator'
      WHEN meta_data.axle = 1 THEN 'Steer'
      ELSE 'Gen 1 Inflator'
    END AS category,
    AVG(cold_inflation_pressure.cip - meta_data.set_point) as deviation_from_set_point
FROM
    {CIP_schema}.cold_inflation_pressure INNER JOIN {schema}.meta_data
        ON meta_data.sensor_number = cold_inflation_pressure.sensor_id
    INNER JOIN {schema}.vehicle_meta_data
        ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
    INNER JOIN {schema}.fleet_meta_data
        ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
    LEFT JOIN {schema}.vehicle_groups_vehicles
        ON vehicle_meta_data.vehicle_id = vehicle_groups_vehicles.vehicle_id
    LEFT JOIN {schema}.vehicle_groups
        ON vehicle_groups_vehicles.vehicle_group_id = vehicle_groups.id
WHERE
    meta_data.id IN (
        SELECT MAX(id)
        FROM {schema}.meta_data
        GROUP BY unique_id
    )
    AND cold_inflation_pressure.time >= date_trunc('year', CURRENT_DATE)::date
    AND fleet_meta_data.code NOT IN {FACs_to_exclude}
    AND fleet_meta_data.status = 'ACTIVE'
GROUP BY
    fleet_meta_data.fleet_name,
    vehicle_groups.name,
    CASE 
        WHEN fleet_meta_data.fleet_name = 'TOMY' THEN 'TOMY'
        ELSE 'AFP'
    END,
    cold_inflation_pressure.time,
    CASE
      WHEN sensor_id ~ '_2$' OR sensor_id ~ '_3$' THEN 'Gen 2 Inflator'
      WHEN meta_data.axle = 1 THEN 'Steer'
      ELSE 'Gen 1 Inflator'
    END
ORDER BY
    date,
    fleet_name, vehicle_group, category
"""
with open("amz_dashboard_CIP_data.txt", "w") as file:
    file.write(query)

In [ ]:
# cip_data = pd.read_sql(query,engine)

# Offline Sensor Reporting

In [12]:
query = f"""
WITH gw_analytics AS (
    SELECT
        vm.vehicle_id,
        MAX(
            CASE
                WHEN gm.type = 'APOLLO' THEN fm.file_upload_time
                WHEN gm.type = 'GEMINI' THEN TIMESTAMP 'epoch' + (hb.event_timestamp / 1000) * INTERVAL '1 second'
            END
        ) AS last_gw_transmission
    FROM {schema}.vehicle_meta_data vm
    INNER JOIN {schema}.gateway_meta_data gm
        ON vm.cycle_number = gm.cycle_number
    LEFT JOIN {schema}.file_meta_data fm
        ON fm.cycle_number = gm.cycle_number
    LEFT JOIN {gemini_schema}.heartbeat hb
        ON hb.device_id = gm.gateway_id
    WHERE fm.id IN (
        SELECT MAX(id) FROM {schema}.file_meta_data GROUP BY cycle_number
    )
    GROUP BY vm.vehicle_id
),
last_tire_transmission AS (
  SELECT 
    meta.vehicle_id,
    meta.side || meta.axle || meta.position AS tire,
    MAX(sd.pressure_date) AS last_sensor_transmission
  FROM {schema}.sensor_data sd 
  INNER JOIN {schema}.meta_data meta
    ON sd.meta_data_id = meta.id
  GROUP BY meta.vehicle_id, meta.side, meta.axle, meta.position
)
SELECT DISTINCT
    fmtd.fleet_name,
    vg.name AS vehicle_group,
    CASE 
      WHEN fmtd.fleet_name = 'TOMY' THEN 'TOMY'
      ELSE 'AFP'
    END AS maint_provider,
    vm.fleet_vehicle_id,
    meta.side || meta.axle || meta.position AS tire,
    meta.sensor_number,
    SA.model_number,
    ls.last_sensor_transmission,
    gw.last_gw_transmission,
    CASE
      WHEN sensor_data.pressure < 4 THEN 'Sensor Removed'
      ELSE 'Unknown'
    END AS sensor_removal_flag,
    sensor_data.pressure AS last_pressure_reading,
    sensor_data.latitude || ' ,' || sensor_data.longitude AS last_known_location
FROM {schema}.vehicle_meta_data vm
INNER JOIN {schema}.meta_data meta
    ON vm.vehicle_id = meta.vehicle_id
LEFT JOIN (SELECT DISTINCT sensor_number, model_number FROM {schema}.sensor_attributes WHERE model_number IS NOT NULL) SA
    ON SA.sensor_number = meta.sensor_number
INNER JOIN last_tire_transmission ls 
    ON vm.vehicle_id = ls.vehicle_id 
       AND (meta.side || meta.axle || meta.position) = ls.tire
INNER JOIN {schema}.sensor_data
    ON sensor_data.meta_data_id = meta.id
    AND sensor_data.pressure_date = ls.last_sensor_transmission
INNER JOIN {schema}.fleet_meta_data fmtd
    ON fmtd.fleet_id = vm.fleet_id
LEFT JOIN {schema}.vehicle_groups_vehicles vgv
    ON vm.vehicle_id = vgv.vehicle_id
LEFT JOIN {schema}.vehicle_groups vg
    ON vgv.vehicle_group_id = vg.id
INNER JOIN gw_analytics gw 
    ON vm.vehicle_id = gw.vehicle_id
WHERE
    (CASE WHEN vm.vehicle_type = 'Tractor Tandem-Dual'
          THEN meta.axle <= 3
          ELSE TRUE END)
    AND vm.archived = 0
    AND ls.last_sensor_transmission < (gw.last_gw_transmission - INTERVAL '2 days')
    AND fmtd.code NOT IN {FACs_to_exclude}
    AND fmtd.status = 'ACTIVE'

"""
with open("amz_dashboard_offline_sensors_query.txt", "w") as file:
    file.write(query)

In [13]:
# df = pd.read_sql(query,engine)

# Vehicles w No Hardware Issues 

In [ ]:
WITH vehicle_list AS (
    SELECT
        fleet_meta_data.fleet_name,
        vehicle_meta_data.vehicle_id,
        vehicle_meta_data.fleet_vehicle_id,
        vehicle_meta_data.vehicle_type
    FROM
        halo_connect_customer_data.fleet_meta_data
    INNER JOIN halo_connect_customer_data.vehicle_meta_data
        ON fleet_meta_data.fleet_id = vehicle_meta_data.fleet_id
    WHERE
        fleet_meta_data.status = 'ACTIVE'
        AND vehicle_meta_data.archived = 0
        AND fleet_meta_data.code NOT IN ('Y87BQV9', 'X77OKO7', '1L10O39N', '1GOPAPMF', '3FF99DS', '1FBR7DGI', '1M5190AC', '19RVO6ZP')
),
sensor_analytics AS (
    SELECT
        vehicle_meta_data.vehicle_id,
        COUNT(
            CASE
                WHEN (CURRENT_TIMESTAMP - MAX(sensor_data.pressure_date)) >= INTERVAL '2 days'
                THEN 1
            END
        ) AS offline_sensors
    FROM
        halo_connect_customer_data.vehicle_meta_data
    INNER JOIN halo_connect_customer_data.meta_data
        ON vehicle_meta_data.vehicle_id = meta_data.vehicle_id
    INNER JOIN halo_connect_customer_data.sensor_data
        ON meta_data.id = sensor_data.meta_data_id
    WHERE
        vehicle_meta_data.archived = 0
    GROUP BY
        vehicle_meta_data.vehicle_id
),
gw_analytics AS (
    SELECT
        vehicle_meta_data.vehicle_id,
        MAX(
            CASE
                WHEN gateway_meta_data.type = 'APOLLO' THEN file_meta_data.file_upload_time
                WHEN gateway_meta_data.type = 'GEMINI' THEN TIMESTAMP 'epoch' + (heartbeat.event_timestamp / 1000) * INTERVAL '1 second'
            END
        ) AS last_gw_transmission
    FROM
        halo_connect_customer_data.vehicle_meta_data
    INNER JOIN halo_connect_customer_data.gateway_meta_data
        ON vehicle_meta_data.cycle_number = gateway_meta_data.cycle_number
    LEFT JOIN halo_connect_customer_data.file_meta_data
        ON file_meta_data.cycle_number = gateway_meta_data.cycle_number
    LEFT JOIN "prod-gem-plat-event-database"."prod-gem-plat-event-table" heartbeat
        ON heartbeat.device_id = gateway_meta_data.gateway_id
    WHERE
        file_meta_data.id IN (
            SELECT MAX(id)
            FROM halo_connect_customer_data.file_meta_data
            GROUP BY cycle_number
        )
    GROUP BY
        vehicle_meta_data.vehicle_id
)
SELECT
    COUNT(DISTINCT vehicle_list.vehicle_id) AS vehicles_with_no_hardware_issues
FROM
    vehicle_list
LEFT JOIN sensor_analytics
    ON vehicle_list.vehicle_id = sensor_analytics.vehicle_id
LEFT JOIN gw_analytics
    ON vehicle_list.vehicle_id = gw_analytics.vehicle_id
WHERE
    COALESCE(sensor_analytics.offline_sensors, 0) = 0
    AND gw_analytics.last_gw_transmission IS NOT NULL
    AND (CURRENT_TIMESTAMP - gw_analytics.last_gw_transmission) < INTERVAL '2 days'
;
